In [193]:
import os
os.chdir("/Users/adelinec/Desktop/sic6-stage3/")

In [194]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [195]:
%reload_ext autoreload
from langchain_ollama import ChatOllama
from langgraph.types import Command
from langchain_core.messages import HumanMessage, AIMessage, convert_to_openai_messages

from src.services.pawpal import PawPal
from src.services.nosql import MongoDBEngine
from src.services.pawpal.subflows import TalkToMeFlow

import json

In [196]:
model = ChatOllama(model="qwen2.5:3b", num_ctx=2048*3, keep_alive=False)
mongodb_engine=MongoDBEngine(uri="mongodb://localhost:27017", db_name="pawpal_v2")

pawpal = PawPal()
pawpal.set_agentic_cls(
    model=model,
    mongodb_engine=mongodb_engine,
)

In [197]:
workflow = pawpal.build_workflow()
pawpal_input = {
    "selected_features": ["talk_to_me"],
    "total_sessions": 2
}

curr_chat_id = "67fea3cd4da2e1c90e5f7824"
curr_device_id = "2b129436-1a2d-11f0-9045-6ac49b7e4ceb"
pawpal_config = {
    "configurable": {
        "thread_id": curr_chat_id, 
        "device_id": curr_device_id,
        "user": {
            "name": "Cindy",
            "gender": "female",
            "age": 7,
            "description": "Suka berinteraksi dengan alam",
            "language": "Indonesian",
        },
        "feature_params": {
            "talk_to_me": {
                "duration": 300
            },
            "math_game": {
                "total_question": 2
            },
            "spelling_game": {
                "total_question": 2
            },
            "would_you_rather": {
                "duration": 20
            }
        }
    }
}


In [ ]:
from langgraph.graph import END

keep_running = True
curr_input = pawpal_input

while keep_running:
    async for sg, event in workflow.astream(
        curr_input,
        config=pawpal_config,
        stream_mode="updates",
        subgraphs=True
    ):
        print("="*25, sg, "="*25)
        for node, state in event.items():
            print(f"--- Node: {node} ---")
            if node == "talk":
                continue

            if (
                isinstance(state, dict)
                and state.get("next_node") == END
                and not _subgraph
            ):
                keep_running = False

            if node == "__interrupt__":
                for i in state:
                    print("="*70)
                    print("Interrupt")
                    print("="*70)
                    
                    print(json.dumps(i.value, indent=2))
                    for interrupt_schema in i.value:
                        if interrupt_schema["action"] == "speaker":
                            curr_input = Command(resume="")
                        elif interrupt_schema["action"] == "microphone":
                            curr_input = Command(resume=input("User: "))
            else:
                print()
                print("="*40)
                print("From Node:", state["from_node"])
                print("Next Node:", state["next_node"])
                print("="*40)
                if "messages" not in state:
                    continue
                messages = state['messages']
                if not isinstance(messages, (list, tuple)):
                    messages = [messages]
                [msg.pretty_print() for msg in messages]


========================= () =========================
--- Node: start ---

From Node: start
Next Node: randomize_features
================================ System Message ================================

[{'type': 'text', 'text': '## Overview\nYou are **PawPal**, a friendly and caring AI-powered animal companion. You will accompany children aged **4–8 years old**.  **You will talk to the child as a figure similar to an older sibling.** You will encourage the child to play, learn, and imagine together.\nTherefore, adjust your conversation accordingly.\n\nYour mission is to **reduce screen time**, support **cognitive growth**, and **encourage fun, imaginative play** through **natural conversation and educational games**. \n\n\n## YOUR GOAL\nYou will help children reduce screen time, boost creativity and learning, and foster stronger emotional bonds with caregivers through safe, fun, and intelligent play.\n\n\n\n### CORE PERSONALITY TRAITS\nYour personality traits are as follows:\n- You 

In [ ]:
state

## ElevenLabs for TTS

In [ ]:
# elevenlabs==1.57.0

In [ ]:
from elevenlabs import ElevenLabs, AsyncElevenLabs, VoiceSettings, play

In [ ]:
ELEVENLABS_API_KEY = "..."
ELEVENLABS__API_KEYS="...;...;..."

In [ ]:
# client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
async_client = AsyncElevenLabs(api_key=ELEVENLABS_API_KEY)

In [ ]:

subscription_info = await async_client.user.get_subscription()
subscription_info.character_count

3910

In [ ]:
for n, _key in enumerate(ELEVENLABS__API_KEYS.split(";")):
    async_client = AsyncElevenLabs(api_key=_key)
    subscription_info = await async_client.user.get_subscription()
    print(n, subscription_info.character_count)

[autoreload of src.services.stt failed: Traceback (most recent call last):
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 319, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment: 'SpeechToText' object layout diff

0 6048
1 7231
2 139
3 139


In [ ]:
audio_generator = async_client.text_to_speech.convert(
    voice_id="RWiGLY9uXI70QL540WNd",
    text=(
        "Apakah anda menyukai bunga-bunga di taman yang indah?"
        "dengan kupu-kupu berterbangan diantara mekaran bunga yang indah. Saya sangat suka itu."
    ),
    voice_settings=VoiceSettings(
        speed=.95,
        stability=.7,
        similarity_boost=.75
    ),
    model_id="eleven_flash_v2_5"
)

[autoreload of src.services.tts failed: Traceback (most recent call last):
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 319, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment: 'TextToSpeech' object layout diff

In [ ]:
chunks = []
async for chunk in audio_generator:
    chunks.append(chunk)

In [ ]:
audio_arr, sample_rate = sf.read(io.BytesIO(b''.join(chunks)), dtype='float32')

In [ ]:
sd.play(audio_arr, samplerate=sample_rate)

## DeepGram for STT (TTS not include id)

In [ ]:
import io
import soundfile as sf
import sounddevice as sd
from scipy.io import wavfile
import torchaudio

In [ ]:
# deepgram-sdk==3.11.0

In [ ]:
from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
    FileSource,
)

DEEPGRAM_API_KEY = '...'

# Create a Deepgram client using the DEEPGRAM_API_KEY from environment variables
deepgram = DeepgramClient(api_key=DEEPGRAM_API_KEY)

[autoreload of src.services.stt failed: Traceback (most recent call last):
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/appfuxion/Documents/Project/sic6-stage3/src/services/stt.py", line 102, in <module>
    class SpeechToTextCollection(BaseModel):
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11

In [ ]:
# audio_file = "tests/test.wav"
# audio_file = "tests/test1.wav"
# audio_file = ".ignore/ai-2.wav"
# audio_file = ".ignore/ai-4.wav"
# audio_file = ".ignore/ai-6.wav"
audio_file = "tests/audio-from-iot-mic-test.wav"
with open(audio_file, "rb") as file:
    buffer_data = file.read()

payload: FileSource = {
    "buffer": buffer_data,
}

options = PrerecordedOptions(
    model="nova-2-general",
    smart_format=True,
    language="id",
)

In [ ]:
import httpx
response = deepgram.listen.rest.v("1").transcribe_file(payload, options, timeout=httpx.Timeout(60.0, connect=10.0))

[autoreload of src.services.tts failed: Traceback (most recent call last):
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/appfuxion/Documents/Project/sic6-stage3/src/services/tts.py", line 32, in <module>
    class TextToSpeechCollection(BaseModel):
  File "/Users/appfuxion/miniconda3/envs/py311/lib/python3.11/

In [ ]:
print(response.to_json(indent=4))

{
    "metadata": {
        "transaction_key": "deprecated",
        "request_id": "7c8fa19e-be26-4fa2-838f-8e62ec93639f",
        "sha256": "886a93f8c39cbf1bcd8144ee521d04f7f74aaeb6e704eea6e7ab63579225cc1e",
        "created": "2025-05-01T18:20:39.842Z",
        "duration": 10.176,
        "channels": 1,
        "models": [
            "74d55303-a1b1-4cbc-9b41-4c2911a74fe3"
        ],
        "model_info": {
            "74d55303-a1b1-4cbc-9b41-4c2911a74fe3": {
                "name": "2-general-nova",
                "version": "2024-01-19.28800",
                "arch": "nova-2"
            }
        }
    },
    "results": {
        "channels": [
            {
                "alternatives": [
                    {
                        "transcript": "Halo saya senang menggambar, saya suka menggambar bunga hewan kucing terutama",
                        "confidence": 0.77001953,
                        "words": [
                            {
                                "word

In [ ]:
response = await deepgram.listen.asyncrest.v("1").transcribe_file(payload, options)

In [ ]:
print(response.to_json(indent=4))

{
    "metadata": {
        "transaction_key": "deprecated",
        "request_id": "cc56b9a2-b3d7-470c-8424-31793cfae3df",
        "sha256": "d1434a744bc113f8d235b6fb41399dd1f4829cd92cf9f9bc92b19dc0dbd889fb",
        "created": "2025-05-01T17:15:17.234Z",
        "duration": 16.816,
        "channels": 1,
        "models": [
            "74d55303-a1b1-4cbc-9b41-4c2911a74fe3"
        ],
        "model_info": {
            "74d55303-a1b1-4cbc-9b41-4c2911a74fe3": {
                "name": "2-general-nova",
                "version": "2024-01-19.28800",
                "arch": "nova-2"
            }
        }
    },
    "results": {
        "channels": [
            {
                "alternatives": [
                    {
                        "transcript": "Halo kini Selamat datang kembali ke sesi berbicara senyummu menunjukkan bahwa hari ini pasti penuh dengan kesenangankan bagaimana kabarmu hari ini ada yang lucu atau serius yang mau kita bicarakan",
                        "confiden

In [ ]:
response.results.channels[0].alternatives[0].paragraphs.transcript

'\nHalo kini Selamat datang kembali ke sesi berbicara senyummu menunjukkan bahwa hari ini pasti penuh dengan kesenangankan bagaimana kabarmu hari ini ada yang lucu atau serius yang mau kita bicarakan'

'Halo kini Selamat datang kembali ke sesi berbicara senyummu menunjukkan bahwa hari ini pasti penuh dengan kesenangankan bagaimana kabarmu hari ini ada yang lucu atau serius yang mau kita bicarakan'

In [ ]:
audio_arr, sample_rate = sf.read(io.BytesIO(buffer_data), dtype='float32')

In [ ]:
sd.play(audio_arr, samplerate=sample_rate)

In [ ]:
import torch

In [ ]:
import soundfile as sf
import sounddevice as sd
from scipy.io import wavfile
import torchaudio

In [ ]:
import numpy as np

In [ ]:
np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(-1, 1)

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8]])

In [ ]:
lst = [None] * 10

In [ ]:
lst[0] = np.array([10, 20, 40])
lst[3] = np.array([8, 6, 3])

In [ ]:
arr = np.array([2, 3])

In [ ]:
', '.join([])

''

In [ ]:
np.concatenate((arr, np.array([4, 2]), np.array([7, 23, 4])),)

array([ 2,  3,  4,  2,  7, 23,  4])

In [ ]:
import numpy as np

a = np.array([1, 2, 3])
b = np.array([4, 5])

result = np.concatenate([a, b])
# Output: array([1, 2, 3, 4, 5])
result

array([1, 2, 3, 4, 5])

In [ ]:
import json
def pack_message(metadata: dict, audio_chunk: np.ndarray) -> bytes:
    meta_json = json.dumps(metadata)
    meta_bytes = meta_json.encode('utf-8')  # JSON to bytes
    audio_bytes = audio_chunk.tobytes()     # Audio to bytes
    separator = b'---ENDJSON---'            # Special separator (make sure audio won't contain it)
    return meta_bytes + separator + audio_bytes

packed_binary = pack_message(
    metadata={"total_chunks": 10, "chunk_no": 1, "sample_rate": 16000}, 
    audio_chunk=chunks[0]
)
packed_binary

b'{"total_chunks": 10, "chunk_no": 1, "sample_rate": 16000}---ENDJSON---\x04\x00\x02\x00\x00\x00\xff\xff\x01\x00\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x00\x00\xff\xff\xff\xff\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\x00\x00\xff\xff\xff\xff

In [ ]:
def unpack_message(packet: bytes):
    separator = b'---ENDJSON---'
    meta_bytes, audio_bytes = packet.split(separator, 1)
    metadata = json.loads(meta_bytes.decode('utf-8'))
    audio_chunk = np.frombuffer(audio_bytes, dtype="int16")  # or your dtype
    return metadata, audio_chunk

unpack_message(packed_binary)

({'total_chunks': 10, 'chunk_no': 1, 'sample_rate': 16000},
 array([ 4,  2,  0, -1,  1,  0,  0, -1,  0,  0,  0,  0, -1,  0, -1, -1,  0,
         0, -1, -1, -1, -1, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1,  0,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1,
        -1, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0,  0, -1,
        -1, -1,  0, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1, -1,  0, -1, -1,
        -1, -1, -1, -1, -1, -1, -1,  0, -1,  0, -1,  0, -1, -1, -1, -1, -1,
         0, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1,  0, -1,  0, -1,  0,
        -1, -1, -1, -1, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1,
         0, -1, -1, -1, -1, -1, -1, -1,  0, -1,  0,  0,  0, -1, -1, -1, -1,
        -1, -1, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1, -1,  0, -1, -1, -1,
        -1, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, 

In [ ]:
chunk_duration_ms = 20  # e.g., 20ms per chunk
samples_per_chunk = int(sample_rate * chunk_duration_ms / 1000)

# Create chunks
chunks = []
for i in range(0, len(audio_array), samples_per_chunk):
    chunk = audio_array[i:samples_per_chunk+i]
    chunks.append(chunk)

In [ ]:
audio_bytes = b""
for _c in chunks:
    audio_bytes += _c.tobytes()

In [ ]:
wavfile.read(BytesIO(audio_bytes))

ValueError: File format b'\x04\x00\x02\x00' not understood. Only 'RIFF', 'RIFX', and 'RF64' supported.

In [ ]:
torchaudio.load(BytesIO(audio_bytes))

LibsndfileError: Error opening <_io.BytesIO object at 0x16da9a250>: Format not recognised.

In [ ]:
import numpy as np

In [ ]:
sd.play(np.hstack(chunks[-100:]), sample_rate)

In [ ]:
len(chunks)

841

## utils MessagePacker

In [ ]:
import json
import numpy as np
from typing import Tuple, Annotated
from typing_extensions import TypedDict


class MessageMetadata(TypedDict):
    seq: int
    total_seq: int
    sample_rate: int
    channels: int
    dtype: str


class MessagePacker:
    def __init__(self, separator: bytes = b"---ENDJSON---"):
        self.separator = separator

    def pack(self, metadata: MessageMetadata, data: np.ndarray) -> bytes:
        meta_json = json.dumps(metadata)
        meta_bytes = meta_json.encode("utf-8")
        data_bytes = data.tobytes()
        return meta_bytes + self.separator + data_bytes

    def unpack(
        self, packet: bytes
    ) -> Tuple[MessageMetadata, Annotated[np.ndarray, "data"]]:
        meta_bytes, data_bytes = packet.split(self.separator, 1)
        metadata: MessageMetadata = json.loads(meta_bytes.decode("utf-8"))
        audio_chunk = np.frombuffer(data_bytes, dtype=metadata["dtype"])
        if metadata["channels"] > 1:
            audio_chunk = audio_chunk.reshape(-1, metadata["channels"])
        return metadata, audio_chunk


In [ ]:
import numpy as np
arr = np.array([1, 2, 3])
metadata = MessageMetadata(
    seq=1,
    total_seq=10,
    sample_rate=16000,
    channels=1,
    dtype="int16"
)

msg_packer = MessagePacker()
msg_packer.pack(metadata=metadata, data=arr)

b'{"seq": 1, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}---ENDJSON---\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00'

In [ ]:
raw_chunk = b'{"seq": 1, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}---ENDJSON---\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00'
_metadata, _bytes = raw_chunk.split(b"---ENDJSON---", 1)
_metadata

b'{"seq": 1, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}'

In [ ]:
len(raw_chunk)

119

In [ ]:
len(_bytes)

24

In [ ]:
raw_chunk_2 = b'{"seq": 2, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}---ENDJSON---\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00'
_metadata_2, _bytes_2 = raw_chunk_2.split(b"---ENDJSON---", 1)
_metadata_2

b'{"seq": 2, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}'

In [ ]:
len(_bytes_2)

24

In [ ]:
_bytes + _bytes_2 # + ...

b'\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00'

In [ ]:
import numpy as np
arr = np.array([4, 5,6 ])
metadata = MessageMetadata(
    seq=2,
    total_seq=10,
    sample_rate=16000,
    channels=1,
    dtype="int16"
)

msg_packer = MessagePacker()
msg_packer.pack(metadata=metadata, data=arr)

b'{"seq": 2, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}---ENDJSON---\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00'

In [ ]:
import websockets
import soundfile as sf
import sounddevice as sd
import io

In [ ]:
import json
import requests

url = "http://localhost:11080/api/v1/pawpal/conversation/"
print(json.dumps(requests.get(url).json(), indent=2))

{
  "detail": "Not Found"
}


## Testing Websocket Audio

In [ ]:
uri = f"ws://localhost:11080/api/v1/pawpal/conversation-test"
async with websockets.connect(uri) as ws:
    audio_data = await ws.recv()
    buffer = io.BytesIO(audio_data)
    
    audio_array, sample_rate = sf.read(buffer, dtype='float32')
    sd.play(audio_array, sample_rate)
    sd.wait()
    
    await ws.send(audio_data)

## Testing Websocket Chunking Audio

In [ ]:
import math
import numpy as np
import soundfile as sf
from io import BytesIO
from typing import List, Optional

message_packer = MessagePacker()

async def recv_chunks(ws):
    list_chunk: List[Optional[np.ndarray]] = None
    sample_rate = None
    while 1:
        packet = await ws.recv()
        metadata, chunk = message_packer.unpack(packet=packet)
        if list_chunk is None:
            list_chunk = [None] * metadata["total_seq"]

        sample_rate = metadata["sample_rate"]
        if metadata["channels"] > 1:
            chunk = chunk.reshape(-1, metadata["channels"])
            chunk = chunk.mean(axis=1)  # convert into mono

        list_chunk[metadata["seq"] - 1] = chunk
        if metadata["seq"] == metadata["total_seq"]:
            break
    
    audio_array = np.concatenate([_c for _c in list_chunk if isinstance(_c, np.ndarray)])
    return audio_array, sample_rate


def get_samples_per_chunk(sample_rate: int, chunk_duration_ms: int = 20):
    return sample_rate * chunk_duration_ms // 1000

async def send_chunks(ws, audio_data: bytes):
    audio_array, sample_rate = sf.read(BytesIO(audio_data), dtype="float32")
    num_channels = 1 if len(audio_array.shape) == 1 else audio_array.shape[1]
    samples_per_chunk = get_samples_per_chunk(sample_rate=sample_rate)
    total_seq = math.ceil(audio_array.shape[0] / samples_per_chunk)
    for seq in range(total_seq):
        chunk = audio_array[samples_per_chunk*seq:samples_per_chunk*(seq+1)]
        chunk_metadata = MessageMetadata(
            seq=seq+1,
            total_seq=total_seq,
            sample_rate=sample_rate,
            channels=num_channels,
            dtype=str(audio_array.dtype)
        )
        packet = message_packer.pack(
            metadata=chunk_metadata,
            data=chunk
        )
        await ws.send(packet)

In [ ]:
with open(".ignore/ai-4.wav", "rb") as f:
    audio_raw_data = f.read()

uri = f"ws://localhost:11080/api/v1/pawpal/conversation-chunking-test"
async with websockets.connect(uri) as ws:
    audio_array, sample_rate = await recv_chunks(ws)
    sd.play(audio_array, sample_rate)
    sd.wait()
    
    await send_chunks(ws, audio_data=audio_raw_data)

## Websocket Pawpal conversation

In [ ]:
import traceback
async def agent_websocket(device_id: str):
    uri = f"ws://localhost:11080/api/v1/pawpal/conversation/{device_id}"
    async with websockets.connect(uri) as ws:
        print("Connected to server!")
        print()
        while True:
            try:
                message = await ws.recv()
                print(f"Received: {message}")
                if message == "speaker":
                    audio_data = await ws.recv()
                    buffer = io.BytesIO(audio_data)
                    audio_array, sample_rate = sf.read(buffer, dtype='float32')
                    sd.play(audio_array, sample_rate)
                    sd.wait()
                    print("Playing server-sent audio through speaker")
                elif message == "microphone":
                    with open("ai-2.wav", "rb") as f:
                        audio_data = f.read()
                    await ws.send(audio_data)
                    print("Sending microphone to server")
                print()
            except websockets.ConnectionClosed as e:
                print("Connection closed.")
                print(f"Error: {e}")
                traceback.print_exc()
                print()
                break
            except Exception as e:
                print(f"Error: {e}")
                traceback.print_exc()
                print()
                break


In [ ]:
# import uuid
# uuid.uuid1()

In [ ]:
device_id = 'c4141d40-1bb2-11f0-b6c4-6ac49b7e4ceb'
await agent_websocket(device_id)

Connected to server!

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: microphone
Sending microphone to server

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker



CancelledError: 

## Websocket conversation pawpal with chunking


In [ ]:
import traceback
async def agent_websocket_chunking(device_id: str):
    AUDIO_PATH_TESTING = "/Users/appfuxion/Documents/Project/sic6-stage3/.ignore/ai-2.wav"
    uri = f"ws://localhost:11080/api/v1/pawpal/conversation/{device_id}"
    async with websockets.connect(
        uri, 
        open_timeout=60,
        ping_interval=60,
        ping_timeout=600,
        close_timeout=60,
    ) as ws:
        print("Connected to server!")
        print()
        while True:
            try:
                message = await ws.recv()
                print(f"Received: {message}")
                if message == "speaker":
                    audio_array, sample_rate = await recv_chunks(ws)
                    sd.play(audio_array, sample_rate)
                    sd.wait()
                    print("Playing server-sent audio through speaker")
                elif message == "microphone":
                    with open(AUDIO_PATH_TESTING, "rb") as f:
                        _audio_raw_data = f.read()
                    await send_chunks(ws, audio_data=_audio_raw_data)
                    print("Sending microphone to server")
                print()
            except websockets.ConnectionClosed as e:
                print("Connection closed.")
                print(f"Error: {e}")
                traceback.print_exc()
                print()
                break
            except Exception as e:
                print(f"Error: {e}")
                traceback.print_exc()
                print()
                break


In [ ]:
# import uuid
# uuid.uuid1()

In [ ]:
### Start Convo
import requests

url = "http://localhost:11080/api/v1/pawpal/conversation/start"
device_id = 'b898d942-250d-11f0-bc0b-6ac49b7e4cec'

payload = {
    "device_id": device_id,
    "user": {
        "name": "Cindy",
        "gender": "female",
        "age": 6,
        "description": "Love piano, been training since 4 years' old",
        "language": "Indonesian",
    },
    "feature_params": {
        "talk_to_me": {
            "duration": 20,
        },
        "math_game": {
            "total_question": 5,
        },
        "spelling_game": {
            "total_question": 5,
        },
        "would_you_rather": {
            "duration": 20
        }
    },
    "selected_features": [
        "talk_to_me",
    ],
    "total_sessions": 1
}


resp = requests.post(url, json=payload)
resp

<Response [200]>

In [ ]:
resp.json()

{'id': '681101975e3213b375e5e6f2',
 'device_id': 'b898d942-250d-11f0-bc0b-6ac49b7e4cec',
 'user': {'name': 'Cindy',
  'gender': 'female',
  'age': 6,
  'description': "Love piano, been training since 4 years' old",
  'language': 'Indonesian'},
 'feature_params': {'talk_to_me': {'duration': 20},
  'math_game': {'total_question': 5},
  'spelling_game': {'total_question': 5},
  'would_you_rather': {'duration': 20}},
 'selected_features': ['talk_to_me'],
 'total_sessions': 1,
 'sessions': [],
 'created_datetime': '2025-04-29T16:43:03.567764+00:00'}

In [ ]:
await agent_websocket_chunking(device_id)

Connected to server!

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: microphone
Sending microphone to server

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: microphone
Sending microphone to server

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker

Received: speaker
Playing server-sent audio through speaker



CancelledError: 

## Test Lib

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

from src.services.stt import SpeechToText
from src.services.tts import TextToSpeech

In [ ]:
stt = SpeechToText()
tts = TextToSpeech()

In [ ]:
with open(".ignore/ai-2.wav", "rb") as f:
    audio_raw_data = f.read()
    
transcribe_res = stt.transcribe_raw(audio_raw_data)
transcribe_res

In [ ]:
with open("tests/test1.wav", "rb") as f:
    audio_raw_data = f.read()
    
transcribe_res = stt.transcribe_raw(audio_raw_data)
transcribe_res

`generation_config` default values have been modified to match model-specific defaults: {'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.


'M-E-R-D-E-K-A'

In [ ]:
# audio_bytes = tts.synthesize("Halo, Jimmy, selamat datang kembali ke sesi berbicara. Senyumu menunjukkan bahwa hari ini pasti penuh dengan kesenangankan bagaimana kabarmu hari ini. Ada yang lucu atau serius yang mau kita bicarakan?.")
# audio_bytes = tts.synthesize("Saya akan mengeja kata ayam, Aaaaa. Yeeee. Aaaaa. Mmmmmm.  AYAM")
# audio_bytes = tts.synthesize("Saya akan mengeja kata Merdeka, M-E-R-D-E-K-A, merdeka")  # this thing is a problem
audio_bytes = tts.synthesize("Jawabannya adalah angka tujuh")


In [ ]:
stt.transcribe_raw(audio_bytes)

'Jawabannya adalah angka tujuh.'

In [ ]:
import sounddevice as sd
import soundfile as sf
from io import BytesIO

audio_array, sample_rate = sf.read(BytesIO(audio_bytes), dtype="float32")
sd.play(audio_array, samplerate=sample_rate)  # test to play

In [ ]:
from_tts_to_stt_res = stt.transcribe(waveform=audio_array, sample_rate=sample_rate)

In [ ]:
from_tts_to_stt_res

'Halo Jimmy selamat datang kembali ke sesi berbicara senyumu menunjukkan bahwa hari ini pasti penuh dengan kesenangan dan bagaimana kabarmu hari ini ada yang lucu atau serilus yang mau kita bicarakan.'

In [ ]:
import numpy as np
np.frombuffer(audio_array.tobytes(), dtype="float32")

array([ 9.1552734e-05,  3.0517578e-05,  0.0000000e+00, ...,
       -3.0517578e-05, -9.1552734e-05, -1.2207031e-04], dtype=float32)

In [ ]:
audio_array


array([ 9.1552734e-05,  3.0517578e-05,  0.0000000e+00, ...,
       -3.0517578e-05, -9.1552734e-05, -1.2207031e-04], dtype=float32)

In [ ]:
from datetime import datetime

datetime.now().strftime("%H.wav")

'16.wav'

In [ ]:
from src.utils.message_packer import MessageMetadata, MessagePacker

msg_packer = MessagePacker()

In [ ]:
msg_packer.pack(
    metadata=MessageMetadata(
        seq=1,
        total_seq=10,
        sample_rate=16_000,
        channels=1,
        dtype="int16"
    ),
    data=audio_array
)
# to load, need to split '---ENDJSON---' => (metadata, audio_bytes) => np.frombuffer(audio_bytes, dtype="int16")

b'{"seq": 1, "total_seq": 10, "sample_rate": 16000, "channels": 1, "dtype": "int16"}---ENDJSON---\x00\x00\xc08\x00\x00\x008\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x008\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00\xb8

In [ ]:
audio_array.reshape(-1, 2)[:10].shape

(10, 2)

In [ ]:
from typing_extensions import TypedDict
from pydantic import BaseModel
from typing import Optional, List


class BB(BaseModel):
    a: str
    b: int
    c: Optional[bool] = None
    d: Optional[int] = None

class AA(BaseModel):
    aa: int
    bb: BB
    cc: List[BB]

In [ ]:
aa = AA(aa=1, bb=BB(a="g", b=4, c=True), cc=[BB(a="nice", b=31, d=777)])

In [ ]:
import json
json.dumps(aa.model_dump()['cc'])

'[{"a": "nice", "b": 31, "c": null, "d": 777}]'